In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'
from sklearn.preprocessing import OneHotEncoder
import time
from sklearn.metrics import silhouette_score

<br>

Load Data

In [2]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train),(x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)
ohe = OneHotEncoder()
y_train_ohe = ohe.fit_transform(y_train.reshape(-1,1)).toarray().astype('float32')
y_test_ohe = ohe.transform(y_test.reshape(-1,1)).toarray().astype('float32')
print('x_train:{}, y_train:{}'.format(x_train.shape, y_train.shape))
print('x_test:{}, y_test:{}'.format(x_test.shape, y_test.shape))
print('y_train_ohe:', y_train_ohe.shape)
print('y_test_ohe:', y_test_ohe.shape)

x_train:(50000, 32, 32, 3), y_train:(50000,)
x_test:(10000, 32, 32, 3), y_test:(10000,)
y_train_ohe: (50000, 10)
y_test_ohe: (10000, 10)


/home/kim1/anaconda3/envs/rok/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


<br>

params

In [3]:
params = {}
params['seed'] = 0
params['embedding'] = 256
params['n_classes'] = 10
params['labels'] = np.unique(y_train).astype('int')
params['batch_size'] = 128
params['logits_scale'] = 10
params['logits_margin'] = 0.1
params['feed_limit'] = 10000

<br>
Model = LeNet5

In [4]:
tf.reset_default_graph()
tf.set_random_seed(params['seed'])

x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y_ohe = tf.placeholder(tf.float32, [None, params['n_classes']])

conv1 = tf.layers.conv2d(inputs=x, filters=6, kernel_size=(5,5), strides=(1,1), padding='valid', name='conv1')
conv1 = tf.nn.relu(conv1)
conv1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=(2,2), strides=(2,2), padding='valid')

conv2 = tf.layers.conv2d(inputs=conv1, filters=16, kernel_size=(5,5), strides=(1,1), padding='valid', name='conv2')
conv2 = tf.nn.relu(conv2)
conv2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=(2,2), strides=(2,2), padding='valid')

flatten = tf.layers.flatten(conv2)

fc1 = tf.layers.dense(flatten, units=120, use_bias=True, activation=tf.nn.relu, name='fc1')
fc2 = tf.layers.dense(fc1, units=84, use_bias=True, activation=tf.nn.relu, name='fc2')
embedding_layer = tf.layers.dense(fc2, units=params['embedding'], use_bias=True, activation=tf.nn.sigmoid)

clf_layer = tf.layers.dense(embedding_layer, units=params['n_classes'], activation=None, use_bias=False, name='clf_layer')
softmax_layer = tf.nn.softmax(clf_layer)

W0830 20:13:30.452948 139864449722112 deprecation.py:323] From <ipython-input-4-820e5c110086>:7: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0830 20:13:30.458902 139864449722112 deprecation.py:506] From /home/kim1/anaconda3/envs/rok/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0830 20:13:30.730068 139864449722112 deprecation.py:323] From <ipython-input-4-820e5c110086>:9: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0830 20:13:30.876702 139864449722112 depr

In [5]:
print('conv1:{}'.format(conv1.shape))
print('conv2:{}'.format(conv2.shape))
print('fc1:{}'.format(fc1.shape))
print('fc2:{}'.format(fc2.shape))
print('embedding_layer:{}'.format(embedding_layer.shape))
print('clf_layer:{}'.format(clf_layer.shape))

conv1:(?, 14, 14, 6)
conv2:(?, 5, 5, 16)
fc1:(?, 120)
fc2:(?, 84)
embedding_layer:(?, 256)
clf_layer:(?, 10)


In [6]:
eta = 1e-4
epsilon = 1e-5
loss = tf.reduce_sum(tf.maximum(tf.multiply(-tf.log(softmax_layer + epsilon), y_ohe), 0))
train_model = tf.train.GradientDescentOptimizer(learning_rate=eta).minimize(loss)

W0830 20:13:31.577783 139864449722112 deprecation.py:323] From /home/kim1/anaconda3/envs/rok/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<br>
Training

In [7]:
loss_path_train = []
loss_path_test = []

embedding_train = []
embedding_test = []

silhouette_train = []
silhouette_test = []

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
iter_cnt = 50
np.random.seed(params['seed'])
sample_size = 10000 # use 10000 samples only for monitoring

# initial embedding train/test 
embedding_train.append(sess.run(embedding_layer, feed_dict={x:x_train[:sample_size]}))
embedding_test.append(sess.run(embedding_layer, feed_dict={x:x_test[:sample_size]})) 

for epoch in range(iter_cnt):
    start_time = time.time()
    cursor = 0
    step = 1
    
    # random shuffle
    train_idx = np.arange(len(y_train))
    np.random.shuffle(train_idx)
    shuffled_x_train = x_train[train_idx]
    shuffled_y_train_ohe = y_train_ohe[train_idx]

    while cursor < len(y_train): 
        batch_x_train = shuffled_x_train[cursor:cursor+params['batch_size']]
        batch_y_train_ohe = shuffled_y_train_ohe[cursor:cursor+params['batch_size']] 
        sess.run(train_model, feed_dict={x:batch_x_train, y_ohe:batch_y_train_ohe})       
        step += 1
        cursor += params['batch_size']
    
    # embedding train/test
    embedding_train.append(sess.run(embedding_layer, feed_dict={x:x_train[:sample_size]}))
    embedding_test.append(sess.run(embedding_layer, feed_dict={x:x_test[:sample_size]}))
    
    # silhouette train/query
    silhouette_train.append(silhouette_score(embedding_train[-1], y_train[:sample_size]))
    silhouette_test.append(silhouette_score(embedding_test[-1], y_test[:sample_size]))    
    
    # loss train/test
    loss_path_train.append(sess.run(loss, feed_dict={x:x_train[:sample_size], y_ohe:y_train_ohe[:sample_size]}))
    loss_path_test.append(sess.run(loss, feed_dict={x:x_test[:sample_size], y_ohe:y_test_ohe[:sample_size]}))
            
    end_time = time.time()
    
    print('epoch:{}    {:.2f}sec \n\
           train(loss:{:.4f}, silhouette:{:.4f}) \n\
           test (loss:{:.4f}, silhouette:{:.4f})'.format(
           epoch+1, end_time - start_time,
           loss_path_train[-1], silhouette_train[-1],
           loss_path_test[-1], silhouette_test[-1]))
    print('')
sess.close()

epoch:1    15.96sec 
           train(loss:23003.7070, silhouette:-0.0503) 
           test (loss:23002.3477, silhouette:-0.0530)

epoch:2    16.38sec 
           train(loss:22912.2559, silhouette:-0.0607) 
           test (loss:22906.1133, silhouette:-0.0615)

epoch:3    15.93sec 
           train(loss:22226.2891, silhouette:-0.0852) 
           test (loss:22203.1562, silhouette:-0.0819)

epoch:4    15.87sec 
           train(loss:21323.6484, silhouette:-0.0930) 
           test (loss:21350.4688, silhouette:-0.0880)

epoch:5    16.16sec 
           train(loss:20036.7500, silhouette:-0.0875) 
           test (loss:20097.5234, silhouette:-0.0861)

epoch:6    16.32sec 
           train(loss:18923.8789, silhouette:-0.0895) 
           test (loss:19010.1895, silhouette:-0.0895)

epoch:7    16.03sec 
           train(loss:18023.0977, silhouette:-0.0758) 
           test (loss:18076.6719, silhouette:-0.0777)

epoch:8    16.55sec 
           train(loss:17173.1172, silhouette:-0.0631) 
       